In [2]:
# import tensorflow as tf
# from transformers import BertTokenizer, TFBertModel
# from tensorflow.keras.layers import Input, Dense, TimeDistributed
# from tensorflow.keras.models import Model
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# from tensorflow.keras.utils import to_categorical
# import pandas as pd
# import numpy as np
# from nltk.corpus import stopwords

# # Load your dataset from the CSV file
# df = pd.read_csv("output.csv", delimiter="\t", names=["input", "key"])
# df['input'] = df['input'].astype(str)
# df['key'] = df['key'].astype(str)

# # Initialize the BERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# # Tokenize and encode sentences for BERT
# X_bert = tokenizer(df['input'].tolist(), padding=True, truncation=True, return_tensors="tf")
# Y_bert = tokenizer(df['key'].tolist(), padding=True, truncation=True, return_tensors="tf")

# # Define the model architecture
# input_ids = Input(shape=(X_bert['input_ids'].shape[1],), dtype='int32')
# bert_output = bert_model(input_ids)[0]
# output = TimeDistributed(Dense(2, activation='softmax'))(bert_output)
# model = Model(inputs=input_ids, outputs=output)

# # Compile and train the model
# y = pad_sequences(Y_bert['input_ids'], padding="post", truncating="post", maxlen=25, value=0)
# y = [to_categorical(i, num_classes=2) for i in y]

# X_train, X_test, y_train, y_test = train_test_split(X_bert['input_ids'], y, test_size=0.20, random_state=42)

# model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
# model.fit(X_train, np.array(y_train), batch_size=32, epochs=5, validation_split=0.1)

# # Save the model
# model.save('bert_keyword_extraction_model.h5')
# tokenizer.save_pretrained('bert_tokenizer')

# # Standard input string
# input_ = "Separate accounts go mainstream investment New entrants are shaking up the separate account industry by supplying web-based platforms that give advisers the tools to pick independent money managers"

# # Tokenize the input string using the trained tokenizer
# input_tokens = tokenizer.texts_to_sequences([input_])
# input_padded = pad_sequences(input_tokens, padding="post", truncating="post", maxlen=25, value=0)

# # Predict the output using the trained model
# output_prediction = model.predict(input_padded)
# keyword_probabilities = output_prediction[0, :, 0]
# top_k_indices = np.where(keyword_probabilities > 0.98)[0]
# top_k_tokens = np.take(input_tokens[0], top_k_indices)
# top_k_keywords = tokenizer.sequences_to_texts([top_k_tokens])[0].split()
# top_k_keywords = [word for word in top_k_keywords if word.lower() not in stopwords.words('english')]
# top_k_keywords = list(set(top_k_keywords))

# print("Top keywords:", top_k_keywords)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

IndexError: index 101 is out of bounds for axis 1 with size 2

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification

# Load your CSV file into a pandas DataFrame
csv_file = "output.csv"
df = pd.read_csv(csv_file)

# Preprocess text data (e.g., remove stopwords, tokenize, etc.)
# Split data into training and testing sets

# Choose a pre-trained BERT model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Change num_labels as needed

# Fine-tune the BERT model on your training data (not shown here)

# Inference
def extract_keywords(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    logits = model(**inputs).logits
    probabilities = torch.softmax(logits, dim=1)
    top_k_indices = torch.topk(probabilities[:, 1], k=5).indices
    keywords = [tokenizer.decode(idx.item()) for idx in top_k_indices]
    return keywords

# Example usage
random_sentence = "This is an example sentence."
keywords = extract_keywords(random_sentence)
print("Top 5 keywords:", keywords)


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the dataset
df = pd.read_csv('output.csv')

# Assume 'input' is the column with text, and 'key' is the column with labels
texts = df['input'].tolist()
labels = df['key'].tolist()

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, encoded_labels, test_size=0.2, random_state=42
)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode texts and labels for BERT
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

class KeywordDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_dataset = KeywordDataset(train_encodings, train_labels)
test_dataset = KeywordDataset(test_encodings, test_labels)


# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()



OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\srini\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load the dataset
df = pd.read_csv('output.csv')

# Assume 'input' is the column with text, and 'key' is the column with labels
texts = df['input'].tolist()
labels = df['key'].tolist()

# Encode the labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Split the dataset into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    texts, encoded_labels, test_size=0.2, random_state=42
)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize and encode texts and labels for BERT
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)

class KeywordDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create the dataset
train_dataset = KeywordDataset(train_encodings, train_labels)
test_dataset = KeywordDataset(test_encodings, test_labels)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# Train the model
trainer.train()


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\srini\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\lib\shm.dll" or one of its dependencies.